<a href="https://colab.research.google.com/github/demoleiwang/SDSC_Bert_Seminar/blob/master/02_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Simple Example for Sentiment Analysis with Bert

If you have any questions, feel free to contact us

Download the IMDB dataset from Google Drive.

In [ ]:
!gdown --id '1vP1lVYFGTLGHjvST3kSH5pxowd_4DcAe' --output IMDB_Dataset.csv

Downloading...
From: https://drive.google.com/uc?id=1vP1lVYFGTLGHjvST3kSH5pxowd_4DcAe
To: /content/IMDB_Dataset.csv
66.2MB [00:02, 31.2MB/s]


In [ ]:
! ls

IMDB_Dataset.csv  sample_data


## Data Processing

Torchtext is a very friendly library for data preparation for PyTorch models. We can use some classic datasets "from torchtext import datasets" directly, such as the IMDB dataset for sentiment analysis or WikiText103 for language modeling. In this example, we do data preparation in a more general way with torchtext, loading data by ourselves.

In [ ]:
! pip install transformers

     |████████████████████████████████| 778kB 9.4MB/s 
     |████████████████████████████████| 3.0MB 30.0MB/s 
     |████████████████████████████████| 1.1MB 52.6MB/s 
     |████████████████████████████████| 890kB 54.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=26fbbe59c8684e7fd750d9c37f3445b06dfc6484bb28f1e31567deb537e67307
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from torchtext import data
from torchtext import datasets

import torch
from transformers import BertTokenizer, BertModel

Load pre-trained bert model and its corresponding tokenizer.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

Considering the special tokens: [cls] and [sep]. we let the maximum length equals to max_length of bert input - 2.  

In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

512


torchtext will make data preparation more easily through predefining data fields.

In [ ]:
REVIEW = data.Field(batch_first=True,
                    use_vocab=False,
                    tokenize = tokenize_and_cut,
                    preprocessing = tokenizer.convert_tokens_to_ids,
                    init_token = tokenizer.cls_token_id,
                    eos_token = tokenizer.sep_token_id,
                    pad_token = tokenizer.pad_token_id,
                    unk_token = tokenizer.unk_token_id
                   )
SENTIMENT = data.LabelField(dtype = torch.float)

In [ ]:
fields = {'review': ('r', REVIEW), 'sentiment': ('s', SENTIMENT)}

Note that loading the data will cost more than 3 minutes.

In [ ]:
IMDB_data = data.TabularDataset(
    path = './IMDB_Dataset.csv',
    format = 'csv',
    fields = fields)

In [ ]:
train_data, valid_data, test_data = IMDB_data.split(split_ratio=[0.8, 0.1, 0.1])

print (len(train_data), len(valid_data), len(test_data))

40000 5000 5000


We apply the bert tokenizer for input text, which contains its own vocab dictionary. Thus, we only need to build vocab for labels.

In [ ]:
SENTIMENT.build_vocab(train_data)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)

cuda


Construct data iterator 

In [ ]:
BATCH_SIZE = 16

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort = False, #don't sort test/validation data
    batch_size=BATCH_SIZE,
    device=device)

## Build a Simple Model with Bert

The model we use refers to [this github repo](https://github.com/bentrevett/pytorch-sentiment-analysis) . It stacks a gru layer over bert.

In [ ]:
import torch.nn as nn


class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):

        super().__init__()

        self.bert = bert

        embedding_dim = bert.config.to_dict()['hidden_size']

        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers=n_layers,
                          bidirectional=bidirectional,
                          batch_first=True,
                          dropout=0 if n_layers < 2 else dropout)

        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        # text = [batch size, sent len]

        with torch.no_grad():
            embedded = self.bert(text)[0]

        # embedded = [batch size, sent len, emb dim]

        _, hidden = self.rnn(embedded)

        # hidden = [n layers * n directions, batch size, emb dim]

        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        else:
            hidden = self.dropout(hidden[-1, :, :])

        # hidden = [batch size, hid dim]

        output = self.out(hidden)

        # output = [batch size, out dim]

        return output

HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

For this exmaple, we fix parameters of bert. It means we only use bert to extract better features for sentiment analysis.

In [ ]:
for name, param in model.named_parameters():
    if name.startswith('bert'):
        param.requires_grad = False

Optimizer and binary loss (binary classification in this case)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

Evaluation function

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

Training and evaluation 

In [ ]:
from tqdm import tqdm

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    with tqdm(total=len(iterator)) as pbar:
      for batch in iterator:
          optimizer.zero_grad()

          input_tensor = batch.r  # .transpose(1,0)
          ground_y = batch.s.squeeze(0)
          #         print (input_tensor.size())
          #         print (ground_y.size())

          predictions = model(input_tensor).squeeze(1)

          loss = criterion(predictions, ground_y)

          acc = binary_accuracy(predictions, ground_y)

          loss.backward()

          optimizer.step()

          epoch_loss += loss.item()
          epoch_acc += acc.item()

          pbar.update(1)

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            input_tensor = batch.r  # .transpose(1,0)
            ground_y = batch.s.squeeze(0)
            #         print (input_tensor.size())
            #         print (ground_y.size())

            predictions = model(input_tensor).squeeze(1)

            loss = criterion(predictions, ground_y)

            acc = binary_accuracy(predictions, ground_y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Better observation for running time 
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Main function

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')

    print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

  0%|          | 0/2500 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 19m 39s
	Train Loss: 0.231 | Train Acc: 90.85%
	 Val. Loss: 0.193 |  Val. Acc: 92.23%


 51%|█████▏    | 1287/2500 [09:15<08:43,  2.32it/s]


KeyboardInterrupt: ignored